# Part 3

With the components on the backwards stepping identified, we now want to undertsand how the sequential unfolding emerges.

## Part 3A
The first answer can be obtained by lookign at information flow or graph cycling at the control level, which we illustrate here.

## Part 3B
We can complement this information using simulations of the neural network.